# Order a Basemap Using the Orders API & SDK via AOI & Quad ID

## Overview ##
---
In this notebook, you will learn how to order a [Planet Basemap](https://developers.planet.com/docs/data/visual-basemaps/) using your [Area of Interest](https://developers.planet.com/apis/orders/basemaps/#order-basemaps-by-area-of-interest-aoi) (AOI) and a [Quad ID](https://developers.planet.com/apis/orders/basemaps/#order-basemaps-by-quad-ids-and-deliver-to-cloud). We will place this order via Planet's [Orders API](https://developers.planet.com/apis/orders/) using our Planet Python [SDK](https://planet-sdk-for-python-v2.readthedocs.io/en/latest/python/sdk-guide/). 

1. Get a Basemap ID using either [Planet Explorer](https://developers.planet.com/docs/apps/explorer/) or the [Basemap Viewer](https://www.planet.com/basemaps/#/mosaic/). 
2. Create a JSON order packet with order parameters.
3. Set up a session with the Planet SDK, and place the order, [delivering]((https://developers.planet.com/apis/orders/delivery/)) the resulting Basemap to a Google Cloud Storage bucket. 
4. Repeat steps 1-3 using quad IDs. 

## Order a Basemap using an AOI
---
### First, find a Basemap to order
You can get a Basemap ID from Planet Explorer or the Basemap Viewer.

#### Option 1: Get a basemap ID from Planet Explorer
You'll need a basemap ID to identify what basemap to download.

To pick a basemap, you can open [Planet Explorer](https://www.planet.com/explorer/#) (you can learn more about Planet Explorer [here](https://developers.planet.com/docs/apps/explorer/)) and select a basemap from the dropdown menu in the lower left corner. (If you don't have access to basemaps, you'll see a message confirming in the lower right corner. Select "Get Access" to sign up for Basemaps or speak to your customer service manager.)

Once you've selected a Basemap, the Explorer updates to include that basemap in the view.

Select the search icon and time cadence, for example `monthly` or `quarterly`. Then select one of the resulting basemap IDs, such as `global_monthly_2022_01_mosaic`. That's the value you'll be passing into the Orders API. 

#### Option 2: Get a basemap ID from the Basemap Viewer

To pick a basemap, you can open the [Basemap Viewer](https://www.planet.com/basemaps/#/mosaic/) (you can learn more about the Basemap Viewer [here](https://developers.planet.com/docs/apps/basemapsviewer/)) and select a basemap from the left sidebar menu, using the filter to narrow down the basemap you want. 

Once you've selected a basemap, and selected the right arrow >, the basemap ID displays, such as `global_monthly_2022_01_mosaic`. That's the value you'll be passing into the Orders API.

### Import Planet and Related Packages

---

Make sure you have Planet's Python SDK properly downloaded. You can find out more about this [here](https://developers.planet.com/docs/pythonclient/). Find your [API key](https://developers.planet.com/quickstart/apis/).

Next set up a session by importing needed Python packages, pulling in your API Key, and make an initial request (to retrieve the Orders API parameters) to confirm a connection with the server.

Create a Google Cloud Platform (GCP) storage bucket with these [instructions](https://cloud.google.com/storage/docs/creating-buckets). This is where we are delivering the data. Creating the bucket will create the credentials for you, and you can convert it to Base64 using these [instructions](https://cloud.google.com/vision/docs/base64) and save this as an environment variable. 

In [257]:
import planet
import os
import copy
import requests
from requests.auth import HTTPBasicAuth
from urllib3.util.retry import Retry
from requests.adapters import HTTPAdapter
import asyncio
import math

In [258]:
PL_API_KEY = os.environ.get('PL_API_KEY')
GCP_CREDENTIALS = os.environ.get('GCP_CREDENTIALS')
auth = planet.Auth.from_key(PL_API_KEY)

### Create an order packet
---
Package up the details of your order in a [JSON object](https://developers.planet.com/apis/orders/basemaps/#example-order-query-json-block) and make a POST request, passing in the Orders URL, your JSON, your API key, and the content-type header. We are [delivering](https://developers.planet.com/apis/orders/delivery/) this order to a Google Cloud Storage bucket. You can see examples of using tools [here](https://developers.planet.com/apis/orders/tools/). Make sure to replace the mosaic name and coordinates with your specifications.

In [259]:
BASEMAP_API_URL = 'https://api.planet.com/basemaps/v1/mosaics'

In [260]:
order_params = {
    "name": "basemap order with geometry",
    "source_type": "basemaps",
    "order_type": "partial",
    "products": [
        {
            "mosaic_name": "global_monthly_2022_01_mosaic",
            "geometry": {
                "type": "Polygon",
                "coordinates": [
                   [
                       [4.607406, 52.353994],
                       [4.680005, 52.353994],
                       [4.680005, 52.395523],
                       [4.607406, 52.395523],
                       [4.607406, 52.353994]
                   ]
                ]
            }
        }
    ],
    "delivery": {
        "google_cloud_storage": {
            "bucket": "devrel-notebooks",
            "credentials": GCP_CREDENTIALS,
            "path_prefix": "basemaps-to-cloud/"
        }
    }
}

### Create a session with SDK and poll for success

Here, we are creating an order using the SDK. Then we are waiting for the order to be delievered to our GCP storage bucket. Note that this can take some time.

In [261]:
async def create_and_deliver_order(order_params, client):
    '''Create an order and wait for it to delivered

    Parameters:
        order_params: An order request
        client: An Order client object
    '''
    with planet.reporting.StateBar(state='creating') as reporter:
        # Place an order to the Orders API
        order = await client.create_order(order_params)
        reporter.update(state='created', order_id=order['id'])
        # Wait while the order is being completed
        await client.wait(order['id'], callback=reporter.update_state,
                          max_attempts=0)

In [262]:
async with planet.Session() as ps:
    # The Orders API client
    client = ps.client('orders')
    # Create the order and deliver it to GCP
    await create_and_deliver_order(order_params, client)

03:25 - order 834961bf-863c-4cfd-9433-359d5d65e392 - state: success


### Check your results

After a few simple steps, we have a basemap delivered to our Google Cloud Platform bucket using the Planet Python SDK.

## Use Basemaps API to get the Basemap and quad IDs
---

### What is a Quad ID? 

A quad ID is a unique identifer to access a square tile of imagery of the Earth's surface, whereas a Basemap is a combination of squares. 

Use the Basemap APIs to retrieve the basemap you want and the quad IDs you are looking for. You can get the base URL you'll need to communicate with the Basemap API service:

1. Go to the [Basemap API reference](https://developers.planet.com/docs/basemaps/reference/) at https://developers.planet.com/docs/basemaps/reference/. 
2. Under List Mosaics, select GET /mosaics.

A dropdown UI element appears with the URL to use:

https://api.planet.com/basemaps/v1/mosaics


#### Ordering basemaps using quad IDs 2 ways: Bounding Box Search and Polygon Search

1. Bounding Box Search: uses a rectangular area, does not require specific geometrical parameters.
2. Polygon Search: searches for quad IDs in a polygonal shape; it is necessary to provide geometrical parameters. 

### Part 1: Bounding Box Search

In [263]:
from requests.adapters import HTTPAdapter
from urllib3.util.retry import Retry

In [264]:
auth = HTTPBasicAuth(PL_API_KEY, '')
session = requests.Session()

# Authenticate
session.auth = (PL_API_KEY, '')

### Handle pagination 
---
It is necessary to always make sure you are reading every page of results as each mosaic can contain up to thousands of quad IDs that would be missed if you do not handle pagination like in the code below. 

In [265]:
def handle_pagination(session, url, params, key='items'):
    """
    Handle paginated URLs by making multiple requests and yielding individual items.

    Parameters:
        session (requests.Session): The session object to make HTTP requests.
        url (str): The base URL for the paginated resource.
        params (dict): Query parameters to be sent with the request.
        key (str, optional): The key in the response JSON containing the list of items. Defaults to 'items'.

    Yields:
        dict: Individual items from the paginated resource.

    Raises:
        requests.HTTPError: If any HTTP errors occur during the requests.

    """
    while True:
        # Make a GET request to the specified URL with the given parameters
        response = session.get(url, params=params)

        # Raise an exception if the response has an HTTP error status code
        response.raise_for_status()

        # Parse the response body as JSON
        body = response.json()

        # Iterate over each item in the 'key' list of the response body and yield it
        for item in body[key]:
            yield item

        # Check if there is a next page link in the response body
        if '_next' in body['_links']:
            # Update the URL to the next page URL
            url = body['_links']['_next']
        else:
            # If there is no next page link, break the loop and stop pagination
            break

### Retrieve quad IDs using Bounding Box Search
---
This code retrieves quad IDs for a specific basemap mosaic. It searches for the quad IDs using the mosaic name and the bounding box parameters.

In [266]:
def bounding_box_search():
    """
    Retrieve quad IDs associated with a specific mosaic from the Planet Basemaps API.

    Returns:
        list: A list of quad IDs.

    """

    # Set the base URL for the Planet Basemaps API
    BASEMAP_API_URL = 'https://api.planet.com/basemaps/v1/mosaics'

    # Wait and retry automatically on 429 "slow down" responses
    retries = Retry(total=10, backoff_factor=1, status_forcelist=[429])
    session.mount('https://', HTTPAdapter(max_retries=retries))

    # Set parameters to filter the mosaic by name
    basemap_params = {
        'name__is': 'fusion_v120rc4_2022-11-04_mosaic'
    }

    session.headers.update({'content-type': 'application/json'})

    # Retrieve the mosaic
    mosaic, = handle_pagination(
        session, BASEMAP_API_URL, basemap_params, 'mosaics')

    # Set parameters for retrieving quads associated with the mosaic using bounding box
    quad_params = {
        'bbox': ','.join(map(str, mosaic['bbox']))
    }

    quads_url = "{}/{}/quads".format(BASEMAP_API_URL, mosaic['id'])

    # Retrieve the quads associated with the mosaic
    quads = handle_pagination(session, quads_url, quad_params)

    # Extract the quad IDs
    quad_ids = [item['id'] for item in quads if 'download' in item['_links']]

    return quad_ids

In [267]:
quad_ids = bounding_box_search()

### Create an order packet
---
Create an order packet with details including the mosaic name and quad IDs, and the tools and delivery method. In this example, we are using the tool [bandmath](https://developers.planet.com/apis/orders/tools/#band-math).

#### Bandmath 

The bandmath tool allows you to apply band math expressions to the bands of your input files to produce derived outputs and indices for analysis. Note that `b5` in this example is being assigned to [NDVI](https://en.wikipedia.org/wiki/Normalized_difference_vegetation_index). 

In [268]:
bounding_box_order_params = {
    "name": "basemap order with quad_ids",
    "source_type": "basemaps",
    "products": [
        {
            "mosaic_name": "fusion_v120rc4_2022-11-04_mosaic",
            "quad_ids": quad_ids
        }
    ],
    "tools": [
        {
            "bandmath": {
                "b1": "b1",
                "b2": "b2",
                "b3": "b3",
                "b4": "b4",
                "b5": "(b4-b3)/(b4+b3)",
                "pixel_type": "32R"
            }
        }
    ],
    "delivery": {
        "google_cloud_storage": {
            "bucket": "devrel-notebooks",
            "credentials": GCP_CREDENTIALS,
            "path_prefix": "basemaps-to-cloud/",
        }
    }
}

### Batch quad IDs and place the orders
---
It is only possible to place orders with 100 quad IDs at a time. This function will batch the quad IDs into lists of 100 each and place each of those orders. 

In [269]:
async def batch_lists_and_place_orders(quad_ids, order_params):
    """
    Process quad IDs in batches and create orders.

    Parameters:
        quad_ids (list): A list of quad IDs to be processed in batches.
        order_params (dict): The order parameters dictionary that contains the details of the order.

    """

    # Calculate the number of batches
    num_batches = math.ceil(len(quad_ids) / 100)

    # Create batched quad IDs lists
    batched_quad_ids = [quad_ids[i:i + 100]
                        for i in range(0, len(quad_ids), 100)]

    # Duplicate the order_params dictionary for each batch
    all_order_params = [copy.deepcopy(order_params)
                        for _ in range(num_batches)]

    # Assign batched quad IDs to each order_params dictionary
    for i, params in enumerate(all_order_params):
        params['products'][0]['quad_ids'] = batched_quad_ids[i]

    async with planet.Session() as ps:
        # The Orders API client
        client = ps.client('orders')

        # Create the order and deliver it to GCP for each batch
        await asyncio.gather(*[
        create_and_deliver_order(params, client) 
        for params in all_order_params
        ])

In [270]:
await batch_lists_and_place_orders(quad_ids, bounding_box_order_params)

07:36 - order 271339da-71d4-473c-83db-d38ca32982ab - state: success
07:51 - order 112a9d88-f601-471d-abdc-d8d52104689c - state: success


### Part 2: Polygon Search

### Define parameters
---
Define the mosaic name and coordinates within the mosoaic below. 

In [271]:
mosaic_name = 'ps_biweekly_sen2_normalized_analytic_subscription_2023-04-17_2023-05-01_mosaic'
geometry = {
    "coordinates": [[[-93.2969252074559, 31.625920464644352],
                     [-91.12907701200592, 31.462828609443292],
                     [-89.55957445650489, 35.19041389553804],
                     [-84.17442809744674, 37.558687870257046],
                     [-90.25210159772769, 37.773797433818345],
                     [-92.93646531283778, 35.481589006307814],
                     [-93.2969252074559, 31.625920464644352]]],
    "type": "Polygon"
}

In [272]:
polygon_search_order_params = {
    "name": "basemap order with quad_ids",
    "source_type": "basemaps",
    "products": [
        {
            "mosaic_name": mosaic_name,
            "quad_ids": quad_ids
        }
    ],
    "tools": [
        {
            "bandmath": {
                "b1": "b1",
                "b2": "b2",
                "b3": "b3",
                "b4": "b4",
                "b5": "(b4-b3)/(b4+b3)",
                "pixel_type": "32R"
            }
        }
    ],
    "delivery": {
        "google_cloud_storage": {
            "bucket": "devrel-notebooks",
            "credentials": GCP_CREDENTIALS,
            "path_prefix": "basemaps-to-cloud/",
        }
    }
}

### Retrieve quad IDs using Polygon Search
---
This function searches for a polygon specified above within a mosaic and returns the quad IDs. 

In [273]:
def polygon_search(mosaic_name, geometry):
    """Searches for quad ID's within a polygon geometry using the Planet Basemaps API.

    Parameters:
        mosaic_name (str): The name of the mosaic to search within.
        geometry (dict): The polygon geometry to search with.

    Yields:
        dict: The quad IDs found within the polygon geometry.

    Raises:
        requests.exceptions.HTTPError: If any HTTP error occurs during the API requests.

    """
    base_url = 'https://api.planet.com/basemaps/v1'

    # Configure retry logic for handling rate limiting (status code 429)
    retries = Retry(total=5, backoff_factor=0.2, status_forcelist=[429])
    session.mount('https://', HTTPAdapter(max_retries=retries))

    # Retrieve the mosaic ID from the mosaic name
    rv = session.get(f'{base_url}/mosaics', params={'name__is': mosaic_name})
    rv.raise_for_status()
    mosaic_id = rv.json()['mosaics'][0]['id']

    url = None
    while True:
        if url is None:
            # Initial request to search for quads within the mosaic
            url = f'{base_url}/mosaics/{mosaic_id}/quads/search'
            rv = session.post(url, json=geometry)
        else:
            # Request subsequent pages of quad search results
            rv = session.get(url)
        rv.raise_for_status()
        response = rv.json()

        # Yield item information for each result item
        for item in response['items']:
            yield item

        # Check if there are more pages of results
        if '_next' in response['_links']:
            url = response['_links']['_next']
        else:
            break

In [274]:
quad_ids = []

# Search for quad IDs and add them to a list.
for quad in polygon_search(mosaic_name, geometry):
    quad_ids.append(quad['id'])

### Place the orders

In [275]:
await batch_lists_and_place_orders(quad_ids, polygon_search_order_params)

00:00 - order  - state: creating



































00:00 - order 89732e12-6c9b-4fa9-b6b3-647fbb8063a0 - state: created









00:00 - order 89732e12-6c9b-4fa9-b6b3-647fbb8063a0 - state: queued 





























































00:05 - order 89732e12-6c9b-4fa9-b6b3-647fbb8063a0 - state: queued



































00:10 - order 89732e12-6c9b-4fa9-b6b3-647fbb8063a0 - state: queued



































00:15 - order 89732e12-6c9b-4fa9-b6b3-647fbb8063a0 - state: running



































00:20 - order 89732e12-6c9b-4fa9-b6b3-647fbb8063a0 - state: running



































00:25 - order 89732e12-6c9b-4fa9-b6b3-647fbb8063a0 - state: running



































00:30 - order 89732e12-6c9b-4fa9-b6b3-647fbb8063a0 - state: running



































00:35 - order 89732e12-6c9b-4fa9-b6b3-647fbb8063a0 - state: running



































00:40 -

### Check your results

After a few simple steps, we have a basemap delivered to our Google Cloud Platform bucket using the Planet Python SDK.  For more information, check out the SDK docs [here](https://planet-sdk-for-python-v2.readthedocs.io/en/latest/python/sdk-guide/). 